# 使用说明

- 登录xuanke网，打开有成绩表的页面，然后保存这个页面到 xuanke_tongji.html。
- 登录4m3后，从浏览器的 cookie 里复制出登录产生的 JSESSION_ID 和 SERVERNAME，填入下方代码中。


Cookie 查看方法是在 Chrome 里打开4m3页面，然后网页上右键，并点“检查” (Inspect)。然后选“应用” (Application)，再选“4m3.tongji.edu.cn”，如图。

注意：JSESSION_ID 一般以 '.62' 结尾，SERVERNAME 以 's' 开头。一个 Session 的有效时间大概在15分钟左右，过期后需要重做上面的步骤以获取新的 Cookie。

- 登录4m3后，查找courseTableForStd!courseTable.action项ids信息，填入下方代码中。

In [36]:
JSESSION_ID = '647C59C3E4E8E08F370083F161A97150.60'
SERVERNAME = 's111'

IDS = 883255830 # IDS与用户选课相关


In [37]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [38]:
xuanke_tongji = BeautifulSoup(open(r'c:\Users\Chenhao\Desktop\xuanke_tongji.html', 'r'), 'html.parser')

In [39]:
# 输出检查一下有没有课程信息，如果没有说明网页没有正确保存下来。
xuanke_tongji


<!-- saved from url=(0110)http://xuanke.tongji.edu.cn/tj_xuankexjgl/score/query/student/cjcx.jsp?qxid=20051013779916&mkid=20051013779901 -->
<html><head><meta content="text/html; charset=utf-8" http-equiv="Content-Type"/><script>
top.HELP_URL="null";
document.onkeydown=function() {
       if(event.srcElement.type!="submit"&&event.srcElement.type!="button"&&event.srcElement.type!="textarea"&&event.srcElement.type!="reset"&&event.keyCode==13){
         event.keyCode=9;
       }

}
function _contextmenu(){
 //return false;
}
document.oncontextmenu=new Function("return _contextmenu();");
</script>
<link href="./xuanke_tongji_files/calendar.css" rel="stylesheet"/>
<link href="./xuanke_tongji_files/main.css" rel="stylesheet"/>
<script src="./xuanke_tongji_files/SuperPublic.js.download"></script>
<script src="./xuanke_tongji_files/DateObject.js.download"></script>
<script src="./xuanke_tongji_files/DatePicker.js.download"></script>
<script src="./xuanke_tongji_files/editlib.js.download"></sc

In [40]:
# 解析 xuanke 网页

table = xuanke_tongji
xuanke_data = pd.DataFrame(columns=['Course_No', 'Course_Title', 'Course_Title_English', 'Year', 'Semester', 'Score', 'Credit', 'Mark', 'Pass', 'Required', 'Update_Time'])

rows = table.find_all('tr')
year = 0
sem = 0



for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    if len(cols) == 1:
        m = re.search(r'(\d+)-(\d+)学年第(\d)', cols[0])
        if m is not None:
            year, _, sem = m.groups(0)
            if sem == '2':
                year = int(year) + 1
        
    if len(cols) == 9 and cols[0] != "课号":
        xuanke_data = xuanke_data.append({
            'Course_No': cols[0],
            'Course_Title': cols[1],
            'Course_Title_English': '',
            'Year': int(year),
            'Semester': 'Fall' if sem == '1' else 'Spring',
            'Score': cols[2],
            'Credit': float(cols[3]),
            'Mark': float(cols[4]),
            'Pass': cols[5],
            'Required': cols[7],
            'Update_Time': cols[8]
        }, ignore_index=True)

print("Found courses in xuanke:", len(xuanke_data))

Found courses in xuanke: 86


In [41]:
# 爬 4m3 网页

course_data = []

print("Finding courses in 4m3...")

for year in ['2015', '2016', '2017', '2018', '2019']: # 这里修改需要爬的年份
    for semester in ['0', '1']:   # 0 = Fall 秋季, 1 = Spring 春季
        
        tongji_semester_id = 100 + 2 * (int(year) - 2015) + int(semester)
        
        page_4m3 = requests.post('http://4m3.tongji.edu.cn/eams/courseTableForStd!courseTable.action',
                      data={'ignoreHead': 1,
                            'setting.kind': 'std',
                            'startWeek': 1,
                            'semester.id': tongji_semester_id,
                            'ids': IDS},
                      cookies={
                        'JSESSIONID': JSESSION_ID,
                        'SERVERNAME': SERVERNAME,
                        'session_locale': 'en_US'
                    })

        soup = BeautifulSoup(page_4m3.content, 'html.parser')
        
        table = soup.find_all('table', attrs={'class':'gridtable'})[1]
        table_body = table.find('tbody')

        rows = table_body.find_all('tr')
        sem_counter = 0
        for row in rows:
            cols = row.find_all('td')

            cols = [ele.text.strip() for ele in cols]
            if cols == []:
                continue
            
            course_no = cols[1][:-2]
            course_name = cols[2].replace('\n', ' ').replace('\t', '').replace('s(', 's (').replace(')(', ') (')
            credit = cols[6]
            course_data.append([year, semester, course_no, course_name, credit])
            sem_counter += 1
        
        print(f"{'Fall' if semester == '0' else 'Spring'} {year}, {sem_counter} courses founded.")

Finding courses in 4m3...
Fall 2015, 14 courses founded.
Spring 2015, 12 courses founded.
Fall 2016, 9 courses founded.
Spring 2016, 9 courses founded.
Fall 2017, 9 courses founded.
Spring 2017, 13 courses founded.
Fall 2018, 8 courses founded.
Spring 2018, 6 courses founded.
Fall 2019, 0 courses founded.
Spring 2019, 1 courses founded.


In [42]:
# 4m3 爬的信息在 course_data 中

In [43]:
for row in course_data:
    course_no = row[2]
    course_name = row[3]
    xuanke_data.loc[xuanke_data.Course_No == course_no, 'Course_Title_English'] = course_name

In [44]:
xuanke_data.to_csv("Transcript.csv", index=False)

In [45]:
xuanke_data

,Course_No,Course_Title,Course_Title_English,Year,Semester,Score,Credit,Mark,Pass,Required,Update_Time
0,002016,形势与政策(1),Current Affairs (1),2015,Fall,优,0.5,5.0,Y,必修,2016-01-19
1,040375,机械与能源工程学导论,Introduction of Mechanical and Energy Engineering,2015,Fall,优,1.0,5.0,Y,必修,2016-01-26
2,100483,大学计算机应用基础,Application Foundation of College Computer,2015,Fall,优,1.5,5.0,Y,必修,2016-01-25
3,110277,大学英语四级,College English (Band 4),2015,Fall,优,2.0,5.0,Y,必修,2016-01-16
4,122004,高等数学(B)上,Advanced Mathematics (B)1,2015,Fall,优,5.0,5.0,Y,必修,2016-01-19
...,...,...,...,...,...,...,...,...,...,...,...
81,040399,机械专业课实验（下）,Experiments for Major Courses (2),2019,Spring,优,0.5,5.0,Y,必修,2019-07-10
82,040458,传热学,Heat Transfer,2019,Spring,优,3.0,5.0,Y,必修,2019-06-26
83,040459,测试与控制技术,,2019,Fall,优,2.0,5.0,Y,必修,2020-04-24
84,040187,毕业设计(论文),,2020,Spring,优,16.0,5.0,Y,必修,2020-06-15
